In [1]:
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]
P4 = [2, 4, 3, 1]
IP = [2, 6, 3, 1, 4, 8, 5, 7]
IP_INV = [4, 1, 3, 5, 7, 2, 8, 6]
EP = [4, 1, 2, 3, 2, 3, 4, 1]
SBOXES = [
    [[1, 0, 3, 2], [3, 2, 1, 0], [0, 2, 1, 3], [3, 1, 3, 2]],
    [[0, 1, 2, 3], [2, 0, 1, 3], [3, 0, 1, 0], [2, 1, 0, 3]]
]

def permute(bits, perm):
    return [bits[i - 1] for i in perm]

def left_shift(bits, shifts):
    return bits[shifts:] + bits[:shifts]

def key_schedule(key):
    key = permute(key, P10)
    left, right = key[:5], key[5:]
    left, right = left_shift(left, 1), left_shift(right, 1)
    k1 = permute(left + right, P8)
    left, right = left_shift(left, 2), left_shift(right, 2)
    k2 = permute(left + right, P8)
    return k1, k2

def sbox_substitution(bits, sbox):
    row = int(f"{bits[0]}{bits[3]}", 2)
    col = int(f"{bits[1]}{bits[2]}", 2)
    return [int(b) for b in f"{SBOXES[sbox][row][col]:02b}"]

def f_function(bits, key):
    bits = permute(bits, EP)
    bits = [b ^ k for b, k in zip(bits, key)]
    left, right = bits[:4], bits[4:]
    sbox_out = sbox_substitution(left, 0) + sbox_substitution(right, 1)
    return permute(sbox_out, P4)

def feistel_round(bits, key):
    left, right = bits[:4], bits[4:]
    f_out = f_function(right, key)
    return right + [l ^ f for l, f in zip(left, f_out)]

def sdes_encrypt(plaintext, key):
    k1, k2 = key_schedule(key)
    bits = permute(plaintext, IP)
    bits = feistel_round(bits, k1)
    bits = bits[4:] + bits[:4]  # Swap halves
    bits = feistel_round(bits, k2)
    return permute(bits, IP_INV)

def sdes_decrypt(ciphertext, key):
    k1, k2 = key_schedule(key)
    bits = permute(ciphertext, IP)
    bits = feistel_round(bits, k2)
    bits = bits[4:] + bits[:4]  # Swap halves
    bits = feistel_round(bits, k1)
    return permute(bits, IP_INV)

# Example usage
key = [0, 1, 0, 0, 0, 0, 1, 1, 1, 0]
plaintext = [1, 0, 1, 0, 1, 0, 1, 0]
ciphertext = sdes_encrypt(plaintext, key)
decrypted_text = sdes_decrypt(ciphertext, key)
print("Ciphertext:", ciphertext)
print("Decrypted:", decrypted_text)

Ciphertext: [1, 0, 1, 1, 1, 0, 1, 0]
Decrypted: [1, 1, 1, 0, 1, 0, 1, 1]
